In [2]:
include("src/sgs_store.jl")

using ..Entity
using ..HllSets

using PyCall
using CSV
using DataFrames
using WordTokenizers

redis   = pyimport("redis")
# Connect to Redis
r = redis.Redis(host="localhost", port=6379, db=0)

csv_file_path = "/home/alexmy/Downloads/POC/DATA/enron_05_17_2015_with_labels_v2.csv"

println(typeof(r))


PyObject


# Selecting tokens by applying multiple filters

In [21]:
matrix = r.fcall("retrieve_tokens", 1, "t:", "refs", "match", "6f983ba3758e7233f7379a9c7b6ee565808a8de6 6bc47f481f9b458cf32e52dbd4d6731a5d198af5", "tf", ">", 4)
# println(matrix[1,1])

17182×2 Matrix{String}:
 "t:2770891406"  "130"
 "t:1315446192"  "8"
 "t:3432805380"  "15"
 "t:2836343235"  "43"
 "t:285191118"   "9"
 "t:2771454975"  "19"
 "t:1741283176"  "6"
 "t:260882023"   "13"
 "t:413436136"   "13"
 "t:3755412643"  "46"
 "t:3405800736"  "47"
 "t:3830922237"  "59"
 "t:3888023939"  "608"
 ⋮               
 "t:556260589"   "12"
 "t:1002174625"  "29"
 "t:3159625224"  "8"
 "t:3581244060"  "5"
 "t:3562180998"  "11"
 "t:1390805360"  "31"
 "t:2713214389"  "5"
 "t:3660487975"  "20"
 "t:4122236966"  "8"
 "t:1007943058"  "5"
 "t:1781610051"  "6"
 "t:2832069688"  "40"

In [ ]:
function generate_overlapping_sets(column, N)
    sets = []
    for i in 1:(length(column) - N + 1)
        push!(sets, column[i:(i + N - 1)])
    end
    return sets
end

first_column = matrix[:, 1]
N = 11

overlapping_sets = generate_overlapping_sets(first_column, N)

# Print the overlapping sets
for set in overlapping_sets
    println(set)
end

# Reading CSV file to DataFrame and formatting some columns

In [14]:
df      = DataFrame(CSV.File(csv_file_path, header=true))
df.From = map(x -> ismissing(x) ? "" : (isnothing(match(r"'([^']*)'", x)) ? "" : match(r"'([^']*)'", x).captures[1]), df.From)
df.To   = map(x -> ismissing(x) ? "" : (isnothing(match(r"'([^']*)'", x)) ? "" : match(r"'([^']*)'", x).captures[1]), df.To)

df_filled = coalesce.(df, "unknown")

print()

In [15]:
"""
    1. Message-ID	
    2. Date	
    3. From	
    4. To	
    5. Subject	
    6. X-From	X-To	X-cc	X-bcc	X-Folder	X-Origin	X-FileName	
    13. content	
    14. user	
    15. Cat_1_level_1	Cat_1_level_2	Cat_1_weight	Cat_2_level_1	Cat_2_level_2	Cat_2_weight	Cat_3_level_1	Cat_3_level_2	Cat_3_weight	Cat_4_level_1	Cat_4_level_2	Cat_4_weight	Cat_5_level_1	Cat_5_level_2	Cat_5_weight	Cat_6_level_1	Cat_6_level_2	Cat_6_weight	Cat_7_level_1	Cat_7_level_2	Cat_7_weight	Cat_8_level_1	Cat_8_level_2	Cat_8_weight	Cat_9_level_1	Cat_9_level_2	Cat_9_weight	Cat_10_level_1	Cat_10_level_2	Cat_10_weight	Cat_11_level_1	Cat_11_level_2	Cat_11_weight	Cat_12_level_1	Cat_12_level_2	Cat_12_weight	
    51. labeled
"""

# Initialize sets to collect data from specific columns
hll_03 = HllSets.HllSet{10}()     # Set{String}()
hll_04 = HllSets.HllSet{10}()     # Set{String}()
hll_05 = HllSets.HllSet{10}()     # Set{String}()
hll_06 = HllSets.HllSet{10}()     # Set{String}()
hll_14 = HllSets.HllSet{10}()     # Set{String}()
hll_15 = HllSets.HllSet{10}()     # Set{String}()


HllSet{10}()


In [16]:
using ..Store
using ..Util
using JSON3
using TextAnalysis
using WordTokenizers
using Base.Threads

tokenizer = WordTokenizers.Words
# r::PyObject, df::DataFrame, parent::String, cols::Vector; p::Int=10, chunk_size::Int=512000
# Store.ingest_df(r, tokenizer, df, csv_file_path, [:From, :To, :Subject, :content, :user])


tokenize (generic function with 1 method)

# Collecting tokens by column and saving processed columns as hash in Redis

In [18]:
cols = [:From, :To, :Subject, :content, :user]
p::Int=10 
chunk_size::Int=512000
_parent = csv_file_path

for column in cols    
    col_values  = df[:, column]
    col_sha1    = Util.sha1_union([_parent, string(column)])
    column_size = Base.summarysize(col_values)
    num_chunks  = ceil(Int, column_size / chunk_size)
    chunks      = Store.chunk_array(col_values, num_chunks)

    println(col_sha1, "; num_chunks: ", num_chunks)
    dataset = Store.ingest_df_column(r, tokenizer, chunks, col_sha1)
    # println(dataset)
    hll = HllSets.HllSet{10}()
    # println(hll)
    dataset = JSON3.write(dataset)
    hll = HllSets.restore(hll, dataset)
    # println(hll)
    entity = Entity.Instance{10}(r, hll)
    println("Column entity instance: ", entity)
end

6f983ba3758e7233f7379a9c7b6ee565808a8de6; num_chunks: 79
Column entity instance: 
Instance(
 sha1: 6f0a114f998daf428bed86d527be629e2cefe8bc
 card: 12173
 hll: HllSet{10}()
 grad: 0.0
 op: nothing)


6bc47f481f9b458cf32e52dbd4d6731a5d198af5; num_chunks: 209
Column entity instance: 
Instance(
 sha1: 3a577c261dc685162db75d6c025fc0fdaf85bf42
 card: 10511
 hll: HllSet{10}()
 grad: 0.0
 op: nothing)


f6c9fedfe796b71638efc125e924040013ef5234; num_chunks: 45
Column entity instance: 
Instance(
 sha1: f92388e6781f4bdf4a9e24e131090233f3bc0dbd
 card: 25811
 hll: HllSet{10}()
 grad: 0.0
 op: nothing)


65875368cc6392683f42a0e2938b5c0789485b97; num_chunks: 1685
Column entity instance: 
Instance(
 sha1: e3ade37139ed79dc97bb646f735bca994fade985
 card: 378258
 hll: HllSet{10}()
 grad: 0.0
 op: nothing)


981f459d81197edf542958361ef219372da6bd82; num_chunks: 5
Column entity instance: 
Instance(
 sha1: 4af3a15f54c2330a7b2e3fe88fc234cfc1dbbbb4
 card: 49
 hll: HllSet{10}()
 grad: 0.0
 op: nothing)




# Building sample set of known size (cardinality) from categorical distribution

In [ ]:
using Random
using Distributions

# Example term frequencies (tf)
terms = ["term1", "term2", "term3", "term4", "term5"]
tf = [10, 20, 30, 25, 15]

# Calculate the total term frequency
total_tf = sum(tf)

# Calculate the probability distribution
probabilities = tf ./ total_tf

# Create a categorical distribution based on the probabilities
term_distribution = Categorical(probabilities)

# Function to sample terms based on the distribution
function sample_terms(terms, term_distribution, num_samples)
    sampled_terms = []
    for _ in 1:num_samples
        term_index = rand(term_distribution)
        push!(sampled_terms, terms[term_index])
    end
    return sampled_terms
end

# Sample 10 terms from the collection
num_samples = 3
sampled_terms = sample_terms(terms, term_distribution, num_samples)

println("Sampled terms: ", sampled_terms)